In [1]:
import pandas as pd
import numpy as np

# Загрузим данные

In [2]:
from sklearn.datasets import load_boston

In [3]:
bunch = load_boston()

In [4]:
#print(bunch.DESCR)

In [5]:
X, y = pd.DataFrame(data=bunch.data, columns=bunch.feature_names.astype(str)), bunch.target

In [6]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Зафиксируем генератор случайных чисел для воспроизводимости:

In [7]:
SEED = 22
np.random.seed = SEED

# Домашка!

Разделим данные на условно обучающую и отложенную выборки:

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [10]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((404, 13), (404,), (102, 13), (102,))

Измерять качество будем с помощью метрики среднеквадратичной ошибки:

In [11]:
from sklearn.metrics import mean_squared_error

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 1.</h3> 
    </div>
    <div class="panel">
        Обучите <b>LinearRegression</b> из пакета <b>sklearn.linear_model</b> на обучающей выборке (<i>X_train, y_train</i>) и измерьте качество на <i>X_test</i>.
        <br>
        <br>
        <i>P.s. Ошибка должна быть в районе 20. </i>
    </div>
</div>

In [12]:
import sklearn.linear_model as lm

In [13]:
clf = lm.LinearRegression().fit(X_train, y_train);
clf.intercept_, clf.coef_;

In [14]:
p = clf.predict(X_test)
p;

In [15]:
mean_squared_error(y_pred=p, y_true=y_test)

20.765767538052035

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 2. (с подвохом)</h3> 
    </div>
    <div class="panel">
        Обучите <b>SGDRegressor</b> из пакета <b>sklearn.linear_model</b> на обучающей выборке (<i>X_train, y_train</i>) и измерьте качество на <i>X_test</i>.
    </div>
</div>

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
ss = StandardScaler()
sgd = lm.SGDRegressor(alpha=0.015, n_iter=200)

X_train_s = ss.fit_transform(X_train)
X_test_s = ss.transform(X_test)

In [18]:
sgd.fit(X_train_s, y_train)
sgd.intercept_, sgd.coef_;

In [19]:
p = sgd.predict(X_test_s)
p;

In [20]:
mean_squared_error(y_pred=p, y_true=y_test)

21.169218541847592

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 3.</h3>
    </div>
    <div class="panel">
        Попробуйте все остальные классы:
        <ul>
            <li>Ridge
            <li>Lasso
            <li>ElasticNet
        </ul>

        <br>

        В них, как вам уже известно, используются параметры регуляризации <b>alpha</b>. Настройте его как с помощью <b>GridSearchCV</b>, так и с помощью готовых <b>-CV</b> классов (<b>RidgeCV</b>, <b>LassoCV</b> и т.д.).
        
        <br><br>

        Найдите уже, в конце-концов, самую точную линейную модель!

    </div>
</div>


In [21]:
model = lm.RidgeCV().fit(X_train, y_train);
#print(model.intercept_, model.coef_)
p = model.predict(X_test)
mean_squared_error(y_pred=p, y_true=y_test)

20.788648064084306

In [22]:
model = lm.LassoCV().fit(X_train, y_train);
#print(model.intercept_, model.coef_)
p = model.predict(X_test)
mean_squared_error(y_pred=p, y_true=y_test)

22.833478633141308

In [23]:
model = lm.ElasticNetCV().fit(X_train, y_train);
#print(model.intercept_, model.coef_)
p = model.predict(X_test)
mean_squared_error(y_pred=p, y_true=y_test)

25.583406340914433

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
params = { 
    'alpha': [10**x for x in range(-3,3)],
}
gs = GridSearchCV(lm.Ridge(), param_grid=params)
gs.fit(X_train, y_train);
#print(model.intercept_, model.coef_)
p = model.predict(X_test)
mean_squared_error(y_pred=p, y_true=y_test)

25.583406340914433

In [26]:
gs.best_params_

{'alpha': 0.1}

<div class="panel panel-info" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задача 4.</h3>
    </div>
    <div class="panel">
        Проверять качество правильно на кросс-валидации, как известно. Вы знаете, что делать: подключаем <b>cross_val_score</b> из <b>sklearn.model_selection</b>. Параметр <b>cv</b> установите равным 5.
        <br><br>
        Вспомните про все штуки, которым мы с вами научились.
        <br><br>
        Добейтесь <b>MSE < 27</b>.
    </div>
</div>

In [27]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [28]:
def make_classifier_with_max_degree(degree):
    return Pipeline([
        ('polynomial_features', PolynomialFeatures(degree=degree)),
        ('scale', StandardScaler()),
        ('lin_regression', lm.Lasso()),
    ])

In [29]:
m = lm.Lasso()
for d in [1,2,3,4,5]:
    clf = make_classifier_with_max_degree(d)
    print('Для полинома степени %s вышла средняя ошибка, равная %5.4f' % \
            (d, -np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))))

Для полинома степени 1 вышла средняя ошибка, равная 31.2555
Для полинома степени 2 вышла средняя ошибка, равная 26.0932
Для полинома степени 3 вышла средняя ошибка, равная 23.3796
Для полинома степени 4 вышла средняя ошибка, равная 21.3855
Для полинома степени 5 вышла средняя ошибка, равная 20.8942
